In [ ]:
!pip install transformers datasets accelerate torch


In [ ]:
from datasets import load_dataset

# MLSUM veri kümesini yükle
dataset = load_dataset("mlsum", "tu")
train_dataset = dataset["train"]


README.md:   0%|          | 0.00/11.0k [00:00<?, ?B/s]

mlsum.py:   0%|          | 0.00/3.72k [00:00<?, ?B/s]

The repository for mlsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mlsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/249277 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11565 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12775 [00:00<?, ? examples/s]

In [ ]:
# İlk 10,000 örneği seç
small_train_dataset = train_dataset.select(range(1000))


In [ ]:
small_validation_dataset = dataset["validation"].select(range(200))  # Örnek: 1000 validation verisi
small_test_dataset = dataset["test"].select(range(100))  # Örnek: 1000 test verisi


In [ ]:
from transformers import AutoTokenizer, AutoModel, Seq2SeqTrainingArguments, Seq2SeqTrainer

model_name = "dbmdz/bert-base-turkish-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/251k [00:00<?, ?B/s]

In [ ]:
model = AutoModel.from_pretrained(model_name)

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

In [ ]:


def split_text_with_sliding_window(text, max_token_length=512, overlap=256):
    """
    Sliding Window yöntemiyle metni parçalara böler.
    :param text: Uzun metin (string)
    :param max_token_length: Modelin işleyebileceği maksimum token sayısı
    :param overlap: Çakışan token sayısı
    :return: Parçalara bölünmüş metinler listesi
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=False)
    input_ids = inputs["input_ids"][0]

    chunks = []
    for start in range(0, len(input_ids), max_token_length - overlap):
        end = min(start + max_token_length, len(input_ids))
        chunks.append(input_ids[start:end])
        if end == len(input_ids):  # Son parçaya ulaşıldığında döngüyü kır
            break

    return chunks

In [ ]:
# Uzun bir metni Sliding Window ile bölme
long_text = small_train_dataset[8]['text']
chunks = split_text_with_sliding_window(long_text)
print(f"Parça Sayısı: {len(chunks)}")


Parça Sayısı: 1


In [ ]:
def preprocess_data(examples):
    inputs = examples['text']
    targets = examples['summary']

    # Tokenize işlemi
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=256, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Eğitim ve doğrulama setlerini işleme
tokenized_train = small_train_dataset.map(preprocess_data, batched=True)
tokenized_val = small_validation_dataset.map(preprocess_data, batched=True)
tokenized_test = small_test_dataset.map(preprocess_data, batched=True)


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from transformers import TrainingArguments, Trainer
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=1,  # GPU belleğine göre ayarlanabilir
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    save_steps=1000,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=500,
    fp16=True,  # Eğer GPU destekliyorsa

)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
trainer.train()


ValueError: The model did not return a loss from the inputs, only the following keys: last_hidden_state,pooler_output. For reference, the inputs it received are input_ids,token_type_ids,attention_mask.

In [ ]:
import torch

# GPU kontrolü
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Modeli ve tokenizer'ı GPU'ya taşı
model = model.to(device)

# Tokenize edilmiş girdiyi GPU'ya taşı
def summarize_with_model(input_text):
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True, max_length=4096)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Tüm tensorları GPU'ya taşı
    outputs = model.generate(**inputs, max_length=512, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test
test_text = small_validation_dataset[4]['text']
predicted_summary = summarize_with_model(test_text)
print("Modelin Özetlediği:")
print(predicted_summary)
